# Example of DOPER - Power Flow

### This examples demonstrates mutli-node power flow model

In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

# Append parent directory to import DOPER
sys.path.append('../src')

### Import DOPER modules

DOPER consists of several modules which are imported here.

In [2]:
from doper import DOPER, get_solver, get_root, standard_report
from doper.models.basemodel import base_model, default_output_list
from doper.models.network import add_network
from doper.models.battery import add_battery
from doper.examples.example import parameter_add_battery, ts_inputs, default_parameter
from doper.plotting import plot_dynamic

### Setup Optimization Model

In [3]:
from pyomo.environ import Objective, minimize

def control_model(inputs, parameter):
    model = base_model(inputs, parameter)
#     model = add_battery(model, inputs, parameter)
    
    model = add_network(model, inputs, parameter)
    
    def objective_function(model):
       return model.sum_energy_cost * parameter['objective']['weight_energy'] \
               + model.sum_demand_cost * parameter['objective']['weight_demand'] \
               + model.sum_export_revenue * parameter['objective']['weight_export'] \
               + model.fuel_cost_total * parameter['objective']['weight_energy'] \
               + model.load_shed_cost_total

    model.objective = Objective(rule=objective_function, sense=minimize, doc='objective function')
    return model

### Define Example Parameter

In [4]:
parameter = default_parameter()
    
# Add nodes and line options
parameter['network'] = {}


# Add network settings to define power-flow constaints
parameter['network']['settings'] = {
    
    # turn off simepl power exchange to utilize full power-flow equations
    'simplePowerExchange': False,
    'simpleNetworkLosses': 0.05,

    # powerflow parameters
    'slackBusVoltage': 1,
    'sBase': 1,
    'vBase': 1,
    'cableDerating': 1,
    'txDerating': 1,

    # power factors
    'powerFactors': {
        'pv': 1,  
        'genset': 1,
        'batteryDisc': 1,
        'batteryChar': 1,
        'load': 1
    },

    # powerflow model settings
    'enableLosses': True,
    'thetaMin': -0.18,
    'thetaMax': 0.09,
    'voltMin': 0.8,
    'voltMax': 1.1,
    'useConsVoltMin': False,
    'enableConstantPf': 1,
    'enableVoltageAngleConstraint': 1,
    'enableGenPqLimits': False, # not implemented yet
}





### Define System Nodes in Paramter

In [5]:
parameter['network']['nodes'] = [ # list of dict to define inputs for each node in network
    { # node 1
        'node_id': 'N1', # unique str to id node
        'pcc': True, # bool to define if node is pcc
        'slack': True, 
        'load_id': None, # str, list of str, or None to find load profile in ts data (if node is load bus) by column label
        'ders': { # dict of der assets at node, if None or not included, no ders present
            'pv_id': None, # str, list, or None to find pv profile in ts data (if pv at node) by column label
            'pv_maxS': 0,
            'battery': None, # list of str corresponding to battery assets (defined in parameter['system']['battery'])
            'genset': None, # list of str correponsing to genset assets (defined in parameter['system']['genset'])
            'load_control': None # str, list or None correponsing to genset assets (defined in parameter['system']['load_control'])
        },
        'connections': [ # list of connected nodes, and line connecting them
            {
                'node': 'N2', # str containing unique node_id of connected node
                'line': 'L1' # str containing unique line_id of line connection nodes, (defined in parameter['network']['lines'])
            },
            {
                'node': 'N4',
                'line': 'L2'
            }
        ]
    },
    { # node 2
        'node_id': 'N2',
        'pcc': True,
        'slack': False, 
        'load_id': 'pf_demand_node2',
        'ders': { 
            'pv_id': 'pf_pv_node2',
            'pv_maxS': 300,
            'battery': 'pf_bat_node2', # node can contain multiple battery assets, so should be list
            'genset': None,
            'load_control': None # node likely to only contain single load_control asset, so should be str
        },
        'connections': [
            {
                'node': 'N1',
                'line': 'L1'
            },
            {
                'node': 'N3',
                'line': 'L1'
            }
        ]
    },
    { # node 3
        'node_id': 'N3',
        'pcc': True,
        'slack': False, 
        'load_id': 'pf_pv_node3',
        'ders': { 
            'pv_id': None,
            'pv_maxS': 1200,
            'battery': 'pf_bat_node3', 
            'genset': 'pf_gen_node3',
            'load_control': None
        },
        'connections': [
            {
                'node': 'N2',
                'line': 'L1'
            }
        ]
    },
    { # node 4
        'node_id': 'N4',
        'pcc': True,
        'slack': False, 
        'load_id': 'pf_demand_node4',
        'ders': { 
            'pv_id': 'pf_pv_node4',
            'pv_maxS': 1000,
            'battery': 'pf_bat_node4', 
            'genset': 'pf_gen_node4',
            'load_control': 'testLc4'
        },
        'connections': [
            {
                'node': 'N1',
                'line': 'L2'
            },
            {
                'node': 'N5',
                'line': 'L3'
            }
        ]
    },
    { # node 5
        'node_id': 'N5',
        'pcc': True,
        'slack': False, 
        'load_id': 'pf_demand_node5',
        'ders': { 
            'pv_id': 'pf_pv_node5',
            'pv_maxS': 1500,
            'battery': 'pf_bat_node5', 
            'genset': 'pf_gen_node5',
            'load_control': None
        },
        'connections': [
            {
                'node': 'N4',
                'line': 'L3'
            }
        ]
    }
]

### Define System Lines in Paramter

In [6]:
parameter['network']['lines'] = [ # list of dicts define each cable/line properties
    {
        'line_id': 'L1',
        'power_capacity': 3500, # line power capacity only used for simple power=exchange
        
        'length': 1200, # line length in meters
        'resistance': 4.64e-6, # line properties are all in pu, based on SBase/VBase defined above
        'inductance': 8.33e-7,
        'ampacity': 3500,
    },
    {
        'line_id': 'L2',
        'power_capacity': 3500,
        
        'length': 1800,
        'resistance': 4.64e-6,
        'inductance': 8.33e-7,
        'ampacity': 3500,
    },
    {
        'line_id': 'L3',
        'power_capacity': 3500,
        
        'length': 900,
        'resistance': 4.64e-6,
        'inductance': 8.33e-7,
        'ampacity': 3500,
    }
]

### Load Example Data

Time-series data are combined to add load and PV generations profiles to each node in the network. Note the column heads used to define these profiles for each node correspond the the 'load_id' and 'pv_id' values used when defining nodes in the 'parameter' dict above.

In [7]:
# create data ts for each node
data2 = ts_inputs(parameter, load='B90', scale_load=700, scale_pv=300)
data3 = ts_inputs(parameter, load='B90', scale_load=1200, scale_pv=1200)
data4 = ts_inputs(parameter, load='B90', scale_load=1500, scale_pv=1000)
data5 = ts_inputs(parameter, load='B90', scale_load=2000, scale_pv=1500)

# use data1 as starting point for multinode df
data = data2.copy()

# drop load and pv from multinode df
data = data.drop(labels='load_demand', axis=1)
data = data.drop(labels='generation_pv', axis=1)

# add node specifc load and pv (where applicable)
data['pf_demand_node2'] = data2['load_demand']
data['pf_demand_node3'] = data3['load_demand']
data['pf_demand_node4'] = data4['load_demand']
data['pf_demand_node5'] = data5['load_demand']

data['pf_pv_node2'] = data2['generation_pv']
data['pf_pv_node3'] = data3['generation_pv']
data['pf_pv_node4'] = data4['generation_pv']
data['pf_pv_node5'] = data5['generation_pv'] 

### Conduct Optimization

In [8]:
# generate standard output data
output_list = default_output_list(parameter)

# Define the path to the solver executable
solver_path = get_solver('cbc', solver_dir=os.path.join(get_root(), 'solvers'))
print(solver_path)
# Initialize DOPER
smartDER = DOPER(model=control_model,
                 parameter=parameter,
                 solver_path=solver_path,
                 output_list=output_list)

# Conduct optimization
res = smartDER.do_optimization(data)

# Get results
duration, objective, df, model, result, termination, parameter = res
print(standard_report(res))

C:\Users\nicholas\.conda\envs\doperDev\lib\site-packages\doper\solvers\Windows64\cbc.exe
Solver			CBC 2.10.3
Duration [s]		197.37
Objective [$]		159216.26			159216.26 (Total Cost)
Cost [$]		7382.64 (Energy)	151833.62 (Demand)
CO2 Emissions [kg]		12670.35



In [9]:
def getVals(model, varName):
    
    vals = getattr(model, varName).extract_values().values()
    n = len(vals)
    
    return {
        'name': varName,
        'sum': int(sum(vals)),
        'mean': int(sum(vals)/float(n)),
        'min': min(vals),
        'max': int(max(vals)),
    }


varList = [
    'load_served_site', 'generation_pv_site',
    'grid_import_site', 'grid_export_site',
    'real_power_inj', 'real_power_abs',
    'imag_power_inj', 'imag_power_abs',
    'electricity_var_provided', 'electricity_var_consumed',
    'electricity_var_purchased'
    ]
        
for vv in varList:
    print(getVals(model, vv))

{'name': 'load_served_site', 'sum': 971362, 'mean': 3506, 'min': 2177.777777777778, 'max': 5400}
{'name': 'generation_pv_site', 'sum': 212142, 'mean': 765, 'min': 0.0, 'max': 2800}
{'name': 'grid_import_site', 'sum': 759463, 'mean': 2741, 'min': 2177.777777777777, 'max': 3811}
{'name': 'grid_export_site', 'sum': 0, 'mean': 0, 'min': 0.0, 'max': 0}
{'name': 'real_power_inj', 'sum': 971605, 'mean': 701, 'min': 0.0, 'max': 3773}
{'name': 'real_power_abs', 'sum': 971362, 'mean': 701, 'min': 0.0, 'max': 1999}
{'name': 'imag_power_inj', 'sum': 0, 'mean': 0, 'min': -12130.71895424836, 'max': 9352}
{'name': 'imag_power_abs', 'sum': 0, 'mean': 0, 'min': 0.0, 'max': 0}
{'name': 'electricity_var_provided', 'sum': 0, 'mean': 0, 'min': -12130.71895424836, 'max': 9352}
{'name': 'electricity_var_consumed', 'sum': 0, 'mean': 0, 'min': 0.0, 'max': 0}
{'name': 'electricity_var_purchased', 'sum': -2733809, 'mean': -1973, 'min': -10000.0, 'max': 9352}
